In [4]:
import boto3
import yaml
import matplotlib.pyplot as plt
import constants as c
import numpy as np
import os
import json
import time
import io
import keras
import tensorflow as tf

from models import ResnetCAE
from tqdm import tqdm
from s3fs.core import S3FileSystem
from tensorflow.keras import layers, Model
from datetime import datetime

with open('secrets.yml', 'r') as f:
    secrets = yaml.safe_load(f)

s3_boto = boto3.client('s3', aws_access_key_id=secrets['aws_access_key_id'], aws_secret_access_key=secrets['aws_secret_access_key'])
bucket  = boto3.resource('s3', aws_access_key_id=secrets['aws_access_key_id'], aws_secret_access_key=secrets['aws_secret_access_key']).Bucket(c.S3_BUCKET)
s3fs    = S3FileSystem(key=secrets['aws_access_key_id'], secret=secrets['aws_secret_access_key'])

In [2]:
read_s3_np = lambda key: np.load(s3fs.open('{}/{}'.format(c.S3_BUCKET, key)))
keys = [d['Key'] for d in s3_boto.list_objects(Bucket=c.S3_BUCKET, Prefix=c.LC_VPATH)['Contents'] if d['Key'].endswith('.npy')]

train_data_int = np.concatenate(list(map(read_s3_np, keys)))
print(f"Loaded {len(keys)} files; resulting stacked array has shape {train_data_int.shape}")

# Convert to one-hot
# We don't preprocess as one-hot because the storage is way larger. 
N, C, H, W = train_data_int.shape
train_data = np.zeros((N, c.LC_N_CLASSES, H, W))

for i in range(c.LC_N_CLASSES):
    train_data[:, i, :, :] = (train_data_int == i).squeeze().astype(int)

# Swap second and fourth axis for Keras
# compatibility
train_data = np.swapaxes(train_data, 1, 3)

Loaded 10 files; resulting stacked array has shape (9999, 1, 32, 32)


In [9]:
# Read args from config file
config_path = "configs/resnet.json"

with open(config_path, 'r') as f:
    config = json.load(f)

    input_shape = config['input_shape']
    K = config['K']
    z_dim = config['z_dim']
    num_filters = config['num_filters']
    n_linear = config['n_linear']

inputs = layers.Input(shape=input_shape)
autoencoder = ResnetCAE(input_shape, K, z_dim, num_filters, n_linear)
outputs = autoencoder(inputs)

model = Model(inputs, outputs)
model.compile(optimizer="adam", loss="categorical_crossentropy")
model.summary()

metadata = {
    "input_shape": input_shape,
    "K": K,
    "z_dim": z_dim,
    "num_filters": num_filters,
    "n_linear": n_linear,
    "summary": model.summary(),
    "loss": "categorical_crossentropy",
}
# Form model ID as timestamp plus parameter count
model_id = f"{int(time.time())}-{len(model.weights)}"

model.fit(train_data, train_data, epochs=1)

Model: "model_86"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_76 (InputLayer)       [(None, 32, 32, 23)]      0         
                                                                 
 resnet_cae_3 (ResnetCAE)    (None, 32, 32, 23)        77975     
                                                                 
Total params: 77975 (304.59 KB)
Trainable params: 77207 (301.59 KB)
Non-trainable params: 768 (3.00 KB)
_________________________________________________________________
Model: "model_86"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_76 (InputLayer)       [(None, 32, 32, 23)]      0         
                                                                 
 resnet_cae_3 (ResnetCAE)    (None, 32, 32, 23)        77975     
                                                                 
To

In [17]:
# Form model ID as timestamp plus parameter count
timestamp_formatted = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
model_id = f"{timestamp_formatted}-{model.count_params()}p"


In [18]:
tjur_r2 = {}
x_true = train_data[0:100]
x_pred = model.predict(x_true)

x_true_int = np.argmax(x_true, axis=-1)
for i in range(c.LC_N_CLASSES):
    is_in_class = (x_true_int == i).any(axis=(1,2))
    x_true_class = x_true[is_in_class]
    x_pred_class = x_pred[is_in_class]
    tjur_r2[i] = (x_true_class * x_pred_class).mean() - ((1-x_true_class) * x_pred_class).mean()

metadata['tjur_r2_class'] = tjur_r2

# Save metadata
metadata_key = f"models/metadata/metadata-{model_id}.json"

# Upload to S3
bucket.put_object(Key=metadata_key, Body=json.dumps(metadata))

4/4 [==============================] - 0s 90ms/step


/var/folders/y6/0j8bg02j23nct6lpnhxhyb4r0000gr/T/ipykernel_64255/4151244415.py:10: RuntimeWarning: Mean of empty slice.
  tjur_r2[i] = (x_true_class * x_pred_class).mean() - ((1-x_true_class) * x_pred_class).mean()


s3.Object(bucket_name='lql-data', key='models/metadata/metadata-2023-08-17-01-45-15-77975p.json')

In [19]:
fig, axes = plt.subplots(6, 2, figsize=(10, 5*6))

for i in range(6):
    axes[i,0].imshow(np.argmax(x_true[i], axis=-1), cmap='tab20b')
    axes[i,1].imshow(np.argmax(x_pred[i], axis=-1), cmap='tab20b')
    axes[i,0].axis('off'), axes[i,1].axis('off')

img_data = io.BytesIO()
plt.savefig(img_data, format='png')
img_data.seek(0)

key = f'figures/reconstructions_{model_id}.png'
bucket.put_object(Body=img_data, ContentType='image/png', Key=key)
plt.close()